### Imports

In [1]:
!pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement apsw==3.33.0.post1
ERROR: No matching distribution found for apsw==3.33.0.post1


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import multiprocessing as mp
import copy
import os

from torchvision.datasets import MNIST

from profilehooks import profile
    
import matplotlib.pylab as plt

#### Get number of threads and choose range of number of clusters

In [4]:
N_PROC = 4 #CHANGE THIS

N_CLUSTER_LOW = 5 #CAN CHANGE THIS
N_CLUSTER_HIGH = 15 #CAN CHANGE THIS

N_INIT = 1 #DON'T CHANGE THIS
N_INIT_TOTAL = 10 #DON'T CHANGE THIS

### Workload

This workload runs K-means clustering on the MNIST dataset for a range of n_cluster values. The idea is to see how well a simple clustering on raw pixel values segments the data since we have labels. There are no embeddings of the data using a neural network and no feature transformations apart from scaling each feature to mean = 0 and standard deviation = 1.

Note that K-means is run on the full dataset for a range of n_cluster values. In addition, for each n_cluster value, by default, scikit-learn's implementation runs the algorithm n_init = 10 times with different random initializations of the cluster locations. Each of these runs is distributed across all available threads by scikit-learn. In addition, we run the range of cluster_values in parallel using multiple threads.

In [5]:
data = MNIST('./', download=True)

In [6]:
#flatten each image
data = data.train_data.numpy().reshape(data.train_data.shape[0], -1).astype(np.float32)
print(data.shape)

(60000, 784)


/home/sanjay/venv_general/lib64/python3.8/site-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [7]:
class Cluster:
    def __init__(self, data,                
                 n_clusters_low=2, 
                 n_clusters_high=50, 
                 n_clusters_stepsize=5, 
                 n_processes=None):

        self.data = data.copy()

        self.data_normalized = None  # zscore -> pca -> zscore

        #clustering scan
        self.n_clusters_low = n_clusters_low
        self.n_clusters_high = n_clusters_high
        self.n_clusters_stepsize = n_clusters_stepsize
        self.n_processes = n_processes

        self.models_dict = None
        self.inertia_dict = None

        #clustering optimal
        self.n_clusters_optimal = None
        self.kmeans_optimal = None

    @profile(immediate=True)
    def train(self):
        self.preprocess_normalize()

        if self.n_processes is None:
            self.find_nclusters()
        else:
            self.find_nclusters_parallel()

        #self.find_elbow()  # hard-coded currently


    def preprocess_normalize(self):        
        self.scaler = StandardScaler()
        self.data_normalized = self.scaler.fit_transform(self.data)

    def find_nclusters(self):
        raise ValueError('You should be passing a value for n_processes')
        
        print('Calling find_nclusters...')
        if self.data_normalized is None:
            raise AttributeError("Please call preprocess_normalize to prepare data for clustering.")

        self.inertia_dict, self.models_dict = {}, {}
        for ncl in range(self.n_clusters_low, self.n_clusters_high, self.n_clusters_stepsize):
            kmeans = KMeans(n_clusters=ncl, n_init=N_INIT, n_jobs=N_PROC)
            kmeans.fit(self.data_normalized)

            self.inertia_dict[ncl] = kmeans.inertia_
            self.models_dict[ncl] = kmeans

    def find_nclusters_parallel(self):
        print('Calling find_nclusters_parallel...')
        manager = mp.Manager()
        models_shared_dict = manager.dict()
        inertia_shared_dict = manager.dict()

        proc_list = []
        counter = 0
        def run(data, ncl, init_id):
            kmeans = KMeans(n_clusters=ncl, n_init=N_INIT)
            kmeans.fit(data)

            inertia_shared_dict[(ncl, init_id)] = kmeans.inertia_
            models_shared_dict[(ncl, init_id)] = kmeans

        for ncl in range(self.n_clusters_low, self.n_clusters_high+1, self.n_clusters_stepsize):
            for init_id in range(N_INIT_TOTAL):
                proc = mp.Process(target=run, args=(self.data_normalized, ncl, init_id))
                proc.start()
                proc_list.append(proc)
                counter += 1

                if counter % self.n_processes == 0:
                    [p.join() for p in  proc_list]
                    proc_list = []

                [p.join() for p in proc_list]

        self.inertia_dict = dict(inertia_shared_dict)
        self.models_dict = dict(models_shared_dict)
    
    def find_best_ninit(self):
        inertia_dict, models_dict = {}, {}

        for ncl in range(N_CLUSTER_LOW, N_CLUSTER_HIGH+1):

            min_inertia_val = self.inertia_dict[(ncl, 0)]
            min_iter_id = 0

            for iter_id in range(1, N_INIT_TOTAL):
                current_inertia_val = self.inertia_dict[(ncl, iter_id)]

                if current_inertia_val < min_inertia_val:
                    min_inertia_val = current_inertia_val
                    min_iter_id = iter_id

            #store best result
            models_dict[ncl] = self.models_dict[(ncl, min_iter_id)]
            inertia_dict[ncl] = self.inertia_dict[(ncl, min_iter_id)]                

            assert(models_dict[ncl].inertia_==inertia_dict[ncl])

        self.models_dict_fine = copy.deepcopy(self.models_dict)
        self.inertia_dict_fine = copy.deepcopy(self.inertia_dict)
        
        self.models_dict = models_dict
        self.inertia_dict = inertia_dict        
    
    def find_elbow(self):
        if self.inertia_dict is None:
            raise AttributeError("Please run find_nclusters to populated inertia_dict.")
        
        keys = np.sort(list(self.inertia_dict.keys()))
        values = np.array([self.inertia_dict[k] for k in keys])

        N = len(keys)

        thresholds = np.arange(2, N-1) #indices to slice at

        threshold_cuts, score_means = [], []

        for t in thresholds:
            model1, model2 = LinearRegression(), LinearRegression()

            domain1 = np.arange(t)
            domain2 = np.arange(t, N)
            
            model1.fit(keys[domain1].reshape(-1,1), values[domain1])
            model2.fit(keys[domain2].reshape(-1,1), values[domain2])

            score1 = model1.score(keys[domain1].reshape(-1,1), values[domain1])
            score2 = model2.score(keys[domain2].reshape(-1,1), values[domain2])

            score = (score1+score2)/2.0
            
            threshold_cuts.append(keys[t])
            score_means.append(score)

        self.n_clusters_optimal = threshold_cuts[np.argmax(score_means)]

        return threshold_cuts, score_means


### Some run times

In [ ]:
cl = Cluster(data, n_clusters_low=N_CLUSTER_LOW, n_clusters_high=N_CLUSTER_HIGH, n_clusters_stepsize=1, n_processes=N_PROC)
cl.train()

Calling find_nclusters_parallel...


In [ ]:
cl.find_best_ninit()

In [ ]:
print(cl.inertia_dict)
print(cl.inertia_dict_fine)

Each column in the table below is a single cluster. The rows show the number of images in a particular cluster that belong to each digit (rows).

E.g.:

Cluster 9 contains both "7" and "9" which are similar in shape

Cluster 2 is dominated by "0"s

Cluster 1 contains "4", "7" and "9" which are also similar in shape

In [ ]:
if 5 in cl.models_dict:
    pred = cl.models_dict[5].predict(cl.data_normalized)
    labels = MNIST('./', download=True).train_labels.numpy()
    
    comp = pd.DataFrame({'clusterid': pred, 'label': labels})
    comp['count'] = 1
    table = pd.pivot_table(comp, index='label', columns='clusterid', values='count', aggfunc='sum')
    
table.fillna(0)

One method to find the optimal number of clusters is to look at the so-called elbow plot of the inertia (sum of distances of each point from its cluster's center) vs the number of clusters. As the number of clusters approaches the number of data points, inertia approaches 0 (since each point is its own clusters). Often, there is a num_cluster values at which the plot shows a dramatic change in slope (slope becomes less negative i.e. increases). This doesn't happen here which is expected since even some variation in an image (pixel-wise translation) can result in a big increase in Euclidean distance but none in the label.

One way to automatically identify the elbow plot is to split the number of clusters axis at different points and fit both the left and right side of the point with linear models. If there's a sharp elbow, both fits should be very good leading to a local maximum of the average R^2 metric. This is implemented in Cluster.find_elbow.

In [ ]:
plt.plot(list(cl.inertia_dict.keys()), list(cl.inertia_dict.values()), 'p')

In [ ]:
threshold_cuts, score_means = cl.find_elbow()

In [ ]:
plt.plot(threshold_cuts, score_means)

In the plot above, the best average $R^2$ score at n_clusters = 10 indicating we should pick 10 as the value for num_clusters.